# Darwin Mini: ZMP trajectory

Learning from Stephane Caron's introductory article: 
https://scaron.info/teaching/prototyping-a-walking-pattern-generator.html

I would like to create a simple program to determine the optimal COM trajectory such that the ZMP contraints are enforced.  If possible, I don't want to depend on `pymanoid`, because it requires OpenRAVE, which is a pain to install (2h to install on an Ubuntu 18.04 VM!)

I found the following optimizer, which seems to be fit my needs:
https://osqp.org/docs/examples/mpc.html

In [2]:
from numpy import array, zeros

In [3]:
class Point(object):

    """
    Point
    
    pos : array, shape=(3,), optional
        Initial position in the world frame.
    vel : array, shape=(3,), optional
        Initial velocity in the world frame.
    accel : array, shape=(3,), optional
        Initial acceleration in the world frame.
    """

    def __init__(self, pos=None, vel=None, accel=None):
        self.__pos = zeros(3) if pos is None else array(pos)
        self.__pd = zeros(3) if vel is None else array(vel)
        self.__pdd = zeros(3) if accel is None else array(accel)

    def copy(self):
        """
        Copy constructor.
        
        color : char, optional
            Color of the copy, in ['r', 'g', 'b'].
        visible : bool, optional
            Should the copy be visible?
        """
        return Point(self.p, self.pd)

    @property
    def p(self):
        """Position coordinates `[x y z]` in the world frame."""
        return self.__pos.copy()

    @property
    def pos(self):
        """Position coordinates `[x y z]` in the world frame."""
        return self.p

    def set_pos(self, pos):
        """
        Set the position of the body in the world frame.
        
        pos : array, shape=(3,)
            3D vector of position coordinates.
        """
        self.__pos = array(pos)

    @property
    def pd(self):
        """Point velocity."""
        return self.__pd.copy()

    def set_vel(self, pd):
        """
        Update the point velocity.
        
        pd : array, shape=(3,)
            Velocity coordinates in the world frame.
        """
        self.__pd = array(pd)

    @property
    def pdd(self):
        """Point acceleration."""
        return self.__pdd.copy()

    def set_accel(self, pdd):
        """
        Update the point acceleration.
        
        pdd : array, shape=(3,)
            Acceleration coordinates in the world frame.
        """
        self.__pdd = array(pdd)

    def integrate_constant_accel(self, pdd, dt):
        """
        Apply Euler integration for a constant acceleration.
        
        pdd : array, shape=(3,)
            Point acceleration in the world frame.
        dt : scalar
            Duration in [s].
        """
        self.set_pos(self.p + (self.pd + .5 * pdd * dt) * dt)
        self.set_vel(self.pd + pdd * dt)
        self.set_accel(pdd)

    def integrate_constant_jerk(self, pddd, dt):
        """
        Apply Euler integration for a constant jerk.
        
        pddd : array, shape=(3,)
            Point jerk in the world frame.
        dt : scalar
            Duration in [s].
        """
        self.set_pos(self.p + dt * (
            self.pd + .5 * dt * (self.pdd + dt * pddd / 3.)))
        self.set_vel(self.pd + dt * (self.pdd + dt * pddd / 2.))
        self.set_accel(self.pdd + dt * pddd)

In [4]:
def generate_footsteps(distance, step_length, foot_spread):
    footsteps = []

    def append_contact(x, y):
        L = 0.088
        w = 0.048
        footsteps.append([(x + L/2, y + w/2), (x + L/2, y - w/2), (x - L/2, y - w/2), (x - L/2, y + w/2)])

    append_contact(0., +foot_spread)
    append_contact(0., -foot_spread)
    x = 0.
    y = foot_spread
    while x < distance:
        if distance - x <= step_length:
            x += min(distance - x, 0.5 * step_length)
        else:  # still some way to go
            x += step_length
        y = -y
        append_contact(x, y)
    append_contact(x, -y)  # now x == distance
    return footsteps

footsteps = generate_footsteps(distance=1.0, step_length=0.1, foot_spread=0.032)

In [5]:
len(footsteps)

14

In [6]:
dt = 0.03  # simulation time step, in [s]
h = 0.088
com = Point([0., 0., h])

In [ ]:
class WalkingFSM(object):

    def __init__(self, ssp_duration, dsp_duration):
        self.dsp_duration = dsp_duration
        self.mpc_timestep = 3 * dt  # update MPC every 90 [ms] (see below)
        self.next_footstep = 2
        self.ssp_duration = ssp_duration
        self.state = None
        self.start_standing()

    def start_standing(self):
        self.start_walking = False
        self.state = "Standing"
        return self.run_standing()

    def run_standing(self):
        if self.start_walking:
            self.start_walking = False
            if self.next_footstep < len(footsteps):
                return self.start_double_support()

    def start_double_support(self):
        if self.next_footstep % 2 == 1:  # left foot swings
            self.stance_foot = stance.right_foot
        else:  # right foot swings
            self.stance_foot = stance.left_foot
        dsp_duration = self.dsp_duration
        if self.next_footstep == 2 or self.next_footstep == len(footsteps) - 1:
            # double support is a bit longer for the first and last steps
            dsp_duration = 4 * self.dsp_duration
        self.swing_target = footsteps[self.next_footstep]
        self.rem_time = dsp_duration
        self.state = "DoubleSupport"
        self.start_com_mpc_dsp()
        return self.run_double_support()

    def run_double_support(self):
        if self.rem_time <= 0.:
            return self.start_single_support()
        self.run_com_mpc()
        self.rem_time -= dt

    def start_com_mpc_dsp(self):
        self.update_mpc(self.rem_time, self.ssp_duration)

    def run_com_mpc(self):
        stance.com.set_x(0.5 * (self.swing_foot.x + self.stance_foot.x))

    def start_single_support(self):
        if self.next_footstep % 2 == 1:  # left foot swings
            self.swing_foot = stance.free_contact('left_foot')
        else:  # right foot swings
            self.swing_foot = stance.free_contact('right_foot')
        self.next_footstep += 1
        self.rem_time = self.ssp_duration
        self.state = "SingleSupport"
        self.start_swing_foot()
        self.start_com_mpc_ssp()
        self.run_single_support()

    def run_single_support(self):
        if self.rem_time <= 0.:
            stance.set_contact(self.swing_foot)
            if self.next_footstep < len(footsteps):
                return self.start_double_support()
            else:  # footstep sequence is over
                return self.start_standing()
        self.run_swing_foot()
        self.run_com_mpc()
        self.rem_time -= dt

    def start_swing_foot(self):
        self.swing_start = self.swing_foot.pose
        self.swing_interp = SwingFoot(
            self.swing_foot, self.swing_target, ssp_duration,
            takeoff_clearance=0.05, landing_clearance=0.05)

    def start_com_mpc_ssp(self):
        self.update_mpc(0., self.rem_time)

    def run_swing_foot(self):
        new_pose = self.swing_interp.integrate(dt)
        self.swing_foot.set_pose(new_pose)

    def on_tick(self, sim):
        if self.state == "Standing":
            return self.run_standing()
        elif self.state == "DoubleSupport":
            return self.run_double_support()
        elif self.state == "SingleSupport":
            return self.run_single_support()
        raise Exception("Unknown state: " + self.state)

    def update_mpc(self, dsp_duration, ssp_duration):
        nb_preview_steps = 16
        T = self.mpc_timestep
        nb_init_dsp_steps = int(round(dsp_duration / T))
        nb_init_ssp_steps = int(round(ssp_duration / T))
        nb_dsp_steps = int(round(self.dsp_duration / T))
        A = array([[1., T, T ** 2 / 2.], [0., 1., T], [0., 0., 1.]])
        B = array([T ** 3 / 6., T ** 2 / 2., T]).reshape((3, 1))
        h = com.p[2]  # z
        g = -9.8  # m/s2
        zmp_from_state = array([1., 0., -h / g])
        C = array([+zmp_from_state, -zmp_from_state])
        D = None
        e = [[], []]
        cur_vertices = self.stance_foot.get_scaled_contact_area(0.8)
        next_vertices = self.swing_target.get_scaled_contact_area(0.8)
        for coord in [0, 1]:
            cur_max = max(v[coord] for v in cur_vertices)
            cur_min = min(v[coord] for v in cur_vertices)
            next_max = max(v[coord] for v in next_vertices)
            next_min = min(v[coord] for v in next_vertices)
            e[coord] = [
                array([+1000., +1000.]) if i < nb_init_dsp_steps else
                array([+cur_max, -cur_min]) if i - nb_init_dsp_steps <= nb_init_ssp_steps else
                array([+1000., +1000.]) if i - nb_init_dsp_steps - nb_init_ssp_steps < nb_dsp_steps else
                array([+next_max, -next_min])
                for i in range(nb_preview_steps)]
        self.x_mpc = LinearPredictiveControl(
            A, B, C, D, e[0],
            x_init=array([stance.com.x, stance.com.xd, stance.com.xdd]),
            x_goal=array([self.swing_target.x, 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.y_mpc = LinearPredictiveControl(
            A, B, C, D, e[1],
            x_init=array([stance.com.y, stance.com.yd, stance.com.ydd]),
            x_goal=array([self.swing_target.y, 0., 0.]),
            nb_steps=nb_preview_steps,
            wxt=1., wu=0.01)
        self.x_mpc.solve()
        self.y_mpc.solve()
        self.preview_time = 0.

    def run_com_mpc(self):
        if self.preview_time >= self.mpc_timestep:
            if self.state == "DoubleSupport":
                self.update_mpc(self.rem_time, self.ssp_duration)
            else:  # self.state == "SingleSupport":
                self.update_mpc(0., self.rem_time)
        com_jerk = array([self.x_mpc.U[0][0], self.y_mpc.U[0][0], 0.])
        stance.com.integrate_constant_jerk(com_jerk, dt)
        self.preview_time += dt


In [1]:
from darwin_mini.trajectory import FootstepTrajectory

In [2]:
traj = FootstepTrajectory(0.1, .005, .05, 0., .088, 'l', 1.)

In [3]:
t1 = [traj(t) for t in [.0, .25, .5, .75, 1.]]

Positions at time 0.000:
  pelvis: (0.025, 0.000, 0.084)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0250, dy=0.0000, dz=0.0336
swing foot vs pelvis:   dx=0.0250, dy=0.0000, dz=0.0336, z_foot=0.0000
Positions at time 0.250:
  pelvis: (0.033, 0.000, 0.086)
  l_foot: (0.016, 0.320, 0.004)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0172, dy=0.0000, dz=0.0317
swing foot vs pelvis:   dx=0.0172, dy=0.0000, dz=0.0279, z_foot=0.0037
Positions at time 0.500:
  pelvis: (0.050, 0.000, 0.088)
  l_foot: (0.050, 0.320, 0.005)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=-0.0000, dy=0.0000, dz=0.0300
swing foot vs pelvis:   dx=-0.0000, dy=0.0000, dz=0.0250, z_foot=0.0050
Positions at time 0.750:
  pelvis: (0.067, 0.000, 0.086)
  l_foot: (0.084, 0.320, 0.004)
  r_foot: (0.050, -0.320, 0.000)
stable foot vs pelvis:  dx=0.0172, dy=0.0000, dz=0.0317
swing foot vs pelvis:   dx=-0.0172, dy=0.0000, dz=0.0279, z_foot=0.0037

In [1]:
from pypot.creatures import DarwinMini

darwin = DarwinMini(simulator='dummy')

In [2]:
from darwin_mini.walk import WalkStraight

In [3]:
w = WalkStraight(darwin, 1.0, 1.0)

w.start()


*** Start step 1 ***
Positions at time 0.000:
  pelvis: (0.000, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.000, -0.320, 0.000)
Positions at time 0.022:
  pelvis: (0.000, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.000, -0.320, 0.001)
Positions at time 0.042:
  pelvis: (0.000, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.000, -0.320, 0.002)
Positions at time 0.062:
  pelvis: (0.000, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.001, -0.320, 0.002)
Positions at time 0.084:
  pelvis: (0.000, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.001, -0.320, 0.003)
Positions at time 0.107:
  pelvis: (0.001, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.002, -0.320, 0.004)
Positions at time 0.132:
  pelvis: (0.001, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.002, -0.320, 0.005)
Positions at time 0.157:
  pelvis: (0.002, 0.000, 0.098)
  l_foot: (0.000, 0.320, 0.000)
  r_foot: (0.003, -0.320, 0.005)
Po

Positions at time 0.506:
  pelvis: (0.050, 0.000, 0.098)
  l_foot: (0.051, 0.320, 0.010)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.529:
  pelvis: (0.052, 0.000, 0.098)
  l_foot: (0.054, 0.320, 0.010)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.552:
  pelvis: (0.054, 0.000, 0.098)
  l_foot: (0.058, 0.320, 0.010)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.574:
  pelvis: (0.056, 0.000, 0.098)
  l_foot: (0.061, 0.320, 0.010)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.599:
  pelvis: (0.057, 0.000, 0.098)
  l_foot: (0.065, 0.320, 0.010)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.624:
  pelvis: (0.059, 0.000, 0.098)
  l_foot: (0.068, 0.320, 0.009)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.646:
  pelvis: (0.061, 0.000, 0.097)
  l_foot: (0.071, 0.320, 0.009)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.671:
  pelvis: (0.062, 0.000, 0.097)
  l_foot: (0.075, 0.320, 0.009)
  r_foot: (0.050, -0.320, 0.000)
Positions at time 0.696:

Positions at time 0.169:
  pelvis: (0.129, 0.000, 0.096)
  l_foot: (0.108, 0.320, 0.006)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.192:
  pelvis: (0.130, 0.000, 0.096)
  l_foot: (0.110, 0.320, 0.006)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.212:
  pelvis: (0.131, 0.000, 0.096)
  l_foot: (0.112, 0.320, 0.007)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.232:
  pelvis: (0.132, 0.000, 0.096)
  l_foot: (0.114, 0.320, 0.007)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.257:
  pelvis: (0.133, 0.000, 0.097)
  l_foot: (0.116, 0.320, 0.008)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.280:
  pelvis: (0.135, 0.000, 0.097)
  l_foot: (0.119, 0.320, 0.008)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.305:
  pelvis: (0.136, 0.000, 0.097)
  l_foot: (0.122, 0.320, 0.008)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.325:
  pelvis: (0.137, 0.000, 0.097)
  l_foot: (0.125, 0.320, 0.009)
  r_foot: (0.150, -0.320, 0.000)
Positions at time 0.350:

Positions at time 0.834:
  pelvis: (0.221, 0.000, 0.096)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.243, -0.320, 0.006)
Positions at time 0.855:
  pelvis: (0.222, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.244, -0.320, 0.005)
Positions at time 0.876:
  pelvis: (0.223, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.246, -0.320, 0.004)
Positions at time 0.896:
  pelvis: (0.224, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.247, -0.320, 0.004)
Positions at time 0.922:
  pelvis: (0.224, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.248, -0.320, 0.003)
Positions at time 0.942:
  pelvis: (0.225, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.249, -0.320, 0.002)
Positions at time 0.963:
  pelvis: (0.225, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.250, -0.320, 0.001)
Positions at time 0.983:
  pelvis: (0.225, 0.000, 0.095)
  l_foot: (0.200, 0.320, 0.000)
  r_foot: (0.250, -0.320, 0.001)

*** Start step 6.0 ***


Positions at time 0.503:
  pelvis: (0.300, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.301, -0.320, 0.010)
Positions at time 0.527:
  pelvis: (0.302, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.304, -0.320, 0.010)
Positions at time 0.552:
  pelvis: (0.304, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.308, -0.320, 0.010)
Positions at time 0.577:
  pelvis: (0.306, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.311, -0.320, 0.010)
Positions at time 0.599:
  pelvis: (0.307, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.315, -0.320, 0.010)
Positions at time 0.621:
  pelvis: (0.309, 0.000, 0.098)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.318, -0.320, 0.009)
Positions at time 0.642:
  pelvis: (0.310, 0.000, 0.097)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.321, -0.320, 0.009)
Positions at time 0.667:
  pelvis: (0.312, 0.000, 0.097)
  l_foot: (0.300, 0.320, 0.000)
  r_foot: (0.324, -0.320, 0.009)
Positions at time 0.691:

...

Positions at time 0.244:
  pelvis: (0.632, 0.000, 0.096)
  l_foot: (0.615, 0.320, 0.007)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.264:
  pelvis: (0.634, 0.000, 0.097)
  l_foot: (0.617, 0.320, 0.008)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.288:
  pelvis: (0.635, 0.000, 0.097)
  l_foot: (0.620, 0.320, 0.008)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.309:
  pelvis: (0.636, 0.000, 0.097)
  l_foot: (0.623, 0.320, 0.009)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.334:
  pelvis: (0.638, 0.000, 0.097)
  l_foot: (0.626, 0.320, 0.009)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.359:
  pelvis: (0.640, 0.000, 0.097)
  l_foot: (0.629, 0.320, 0.009)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.384:
  pelvis: (0.641, 0.000, 0.098)
  l_foot: (0.633, 0.320, 0.009)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.408:
  pelvis: (0.643, 0.000, 0.098)
  l_foot: (0.636, 0.320, 0.010)
  r_foot: (0.650, -0.320, 0.000)
Positions at time 0.431:

Positions at time 0.931:
  pelvis: (0.724, 0.000, 0.095)
  l_foot: (0.700, 0.320, 0.000)
  r_foot: (0.749, -0.320, 0.003)
Positions at time 0.953:
  pelvis: (0.725, 0.000, 0.095)
  l_foot: (0.700, 0.320, 0.000)
  r_foot: (0.749, -0.320, 0.002)
Positions at time 0.976:
  pelvis: (0.725, 0.000, 0.095)
  l_foot: (0.700, 0.320, 0.000)
  r_foot: (0.750, -0.320, 0.001)

*** Start step 16.0 ***
Positions at time 0.001:
  pelvis: (0.725, 0.000, 0.095)
  l_foot: (0.700, 0.320, 0.000)
  r_foot: (0.750, -0.320, 0.000)
Positions at time 0.026:
  pelvis: (0.725, 0.000, 0.095)
  l_foot: (0.700, 0.320, 0.001)
  r_foot: (0.750, -0.320, 0.000)
Positions at time 0.049:
  pelvis: (0.725, 0.000, 0.095)
  l_foot: (0.701, 0.320, 0.002)
  r_foot: (0.750, -0.320, 0.000)
Positions at time 0.075:
  pelvis: (0.726, 0.000, 0.095)
  l_foot: (0.702, 0.320, 0.003)
  r_foot: (0.750, -0.320, 0.000)
Positions at time 0.095:
  pelvis: (0.726, 0.000, 0.095)
  l_foot: (0.703, 0.320, 0.003)
  r_foot: (0.750, -0.320, 0.000)

Positions at time 0.609:
  pelvis: (0.808, 0.000, 0.098)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.816, -0.320, 0.010)
Positions at time 0.632:
  pelvis: (0.810, 0.000, 0.098)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.819, -0.320, 0.009)
Positions at time 0.657:
  pelvis: (0.811, 0.000, 0.097)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.823, -0.320, 0.009)
Positions at time 0.678:
  pelvis: (0.813, 0.000, 0.097)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.826, -0.320, 0.009)
Positions at time 0.703:
  pelvis: (0.814, 0.000, 0.097)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.829, -0.320, 0.008)
Positions at time 0.728:
  pelvis: (0.816, 0.000, 0.097)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.832, -0.320, 0.008)
Positions at time 0.753:
  pelvis: (0.817, 0.000, 0.096)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.835, -0.320, 0.007)
Positions at time 0.774:
  pelvis: (0.819, 0.000, 0.096)
  l_foot: (0.800, 0.320, 0.000)
  r_foot: (0.837, -0.320, 0.007)
Positions at time 0.796:

Positions at time 0.273:
  pelvis: (0.884, 0.000, 0.097)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.868, -0.320, 0.008)
Positions at time 0.295:
  pelvis: (0.885, 0.000, 0.097)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.871, -0.320, 0.008)
Positions at time 0.315:
  pelvis: (0.887, 0.000, 0.097)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.874, -0.320, 0.009)
Positions at time 0.338:
  pelvis: (0.888, 0.000, 0.097)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.877, -0.320, 0.009)
Positions at time 0.362:
  pelvis: (0.890, 0.000, 0.097)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.880, -0.320, 0.009)
Positions at time 0.387:
  pelvis: (0.892, 0.000, 0.098)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.883, -0.320, 0.009)
Positions at time 0.412:
  pelvis: (0.893, 0.000, 0.098)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.887, -0.320, 0.010)
Positions at time 0.437:
  pelvis: (0.895, 0.000, 0.098)
  l_foot: (0.900, 0.320, 0.000)
  r_foot: (0.891, -0.320, 0.010)
Positions at time 0.459:

Positions at time 0.947:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (0.999, 0.320, 0.002)
  r_foot: (0.950, -0.320, 0.000)
Positions at time 0.969:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.001)
  r_foot: (0.950, -0.320, 0.000)
Positions at time 0.993:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.950, -0.320, 0.000)

*** Start step 21.0 ***
Positions at time 0.014:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.950, -0.320, 0.001)
Positions at time 0.039:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.950, -0.320, 0.002)
Positions at time 0.064:
  pelvis: (0.975, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.951, -0.320, 0.002)
Positions at time 0.089:
  pelvis: (0.976, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.951, -0.320, 0.003)
Positions at time 0.114:
  pelvis: (0.976, 0.000, 0.095)
  l_foot: (1.000, 0.320, 0.000)
  r_foot: (0.952, -0.320, 0.004)

In [5]:
darwin

<Robot motors=[<DxlMotor name=neck_joint id=17 pos=0.0>, <DxlMotor name=l_shoulder_joint id=2 pos=0.0>, <DxlMotor name=l_biceps_joint id=4 pos=0.0>, <DxlMotor name=l_elbow_joint id=6 pos=0.0>, <DxlMotor name=r_shoulder_joint id=1 pos=0.0>, <DxlMotor name=r_biceps_joint id=3 pos=0.0>, <DxlMotor name=r_elbow_joint id=5 pos=0.0>, <DxlMotor name=l_hip_joint id=8 pos=0.0>, <DxlMotor name=l_thigh_joint id=10 pos=38.01483718241529>, <DxlMotor name=l_knee_joint id=12 pos=-79.32806983704167>, <DxlMotor name=l_ankle_joint id=14 pos=41.31323265462636>, <DxlMotor name=l_foot_joint id=16 pos=0.0>, <DxlMotor name=r_hip_joint id=7 pos=0.0>, <DxlMotor name=r_thigh_joint id=9 pos=-37.51851384746651>, <DxlMotor name=r_knee_joint id=11 pos=78.22617518208934>, <DxlMotor name=r_ankle_joint id=13 pos=-40.70766133462282>, <DxlMotor name=r_foot_joint id=15 pos=0.0>]>